In [ ]:
!pip install PyMuPDF openai


In [ ]:
import fitz  # PyMuPDF
import openai
import random

In [ ]:
def convert_pdf_to_text(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Function to generate QA pairs using GPT-4
def generate_qa_pairs(context, prompt):
    response = openai.Completion.create(
        engine="gpt-4",
        prompt=prompt.format(context=context),
        max_tokens=150,
        n=1,
        stop=None,
        temperature=0.7
    )
    return response.choices[0].text.strip()

# Function to create prompts for GPT-4
def create_prompts(context):
    unique_prompt = "Generate a unique question-answer pair based on the following context:\n\nContext: {context}\n\nQ: "
    repeated_prompt = "Generate different answers for the same question based on the following context:\n\nContext: {context}\n\nQ: "
    return unique_prompt, repeated_prompt



In [ ]:
# Generate QA pairs from PDF
def generate_qa_pairs_from_pdf(pdf_path, total_pairs, repeated_pairs, unique_pairs):
    text = convert_pdf_to_text(pdf_path)
    context_samples = text.split('\n\n')  # Simple split, can be refined
    
    # Ensure enough context samples
    if len(context_samples) < total_pairs:
        raise ValueError("Not enough context samples in the provided PDF.")
    
    qa_pairs = []
    used_contexts = random.sample(context_samples, repeated_pairs + unique_pairs)
    
    for i in range(unique_pairs):
        context = used_contexts[i]
        unique_prompt, _ = create_prompts(context)
        qa_pair = generate_qa_pairs(context, unique_prompt)
        qa_pairs.append({"context": context, "qa_pair": qa_pair, "type": "unique"})
    
    for i in range(repeated_pairs):
        context = used_contexts[unique_pairs + i]
        _, repeated_prompt = create_prompts(context)
        base_qa_pair = generate_qa_pairs(context, repeated_prompt)
        for j in range(2):  # Generate two different answers for the same question
            different_answer = generate_qa_pairs(context, repeated_prompt + base_qa_pair + "\nA: ")
            qa_pairs.append({"context": context, "qa_pair": base_qa_pair + "\nA: " + different_answer, "type": "repeated"})
    
    return qa_pairs


In [ ]:
# Example usage
pdf_path = "data/defining nursing_2003.pdf"
total_pairs = 500
repeated_pairs = 125
unique_pairs = 125

qa_pairs = generate_qa_pairs_from_pdf(pdf_path, total_pairs, repeated_pairs, unique_pairs)



In [ ]:
# Display some QA pairs
for pair in qa_pairs[:10]:  # Display first 10 pairs
    print(pair)